In [ ]:
import pandas as pd
import numpy as np 
from sklearn.impute import KNNImputer

dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv') 

In [ ]:
#drop type 'object'
dfTrain_drop_obj = dfTrain.select_dtypes(exclude=['object'])
dfTest_drop_obj = dfTest.select_dtypes(exclude=['object'])

TrainY_drop_obj = dfTrain_drop_obj.loc[:,'target'].values
TrainX_drop_obj = dfTrain_drop_obj.loc[:,'f2':'f290'].values
TestX_drop_obj = dfTest_drop_obj.loc[:,'f2':].values

# print total missing
print('Missing(TRAIN): %d' % sum(np.isnan(TrainX_drop_obj).flatten()))
print('Missing(TEST): %d' % sum(np.isnan(TestX_drop_obj).flatten()))
# define imputer
imputerTrain = KNNImputer()
imputerTest  = KNNImputer()
# fit on the dataset
imputerTrain.fit(TrainX_drop_obj)
imputerTest.fit(TestX_drop_obj)
# transform the dataset
Xtrain = imputerTrain.transform(TrainX_drop_obj)
Xtest = imputerTest.transform(TestX_drop_obj)
# print total missing
print('Missing(TRAIN): %d' % sum(np.isnan(Xtrain).flatten()))
print('Missing(TEST): %d' % sum(np.isnan(Xtest).flatten()))

np.savetxt("Ytrain_drop_obj.csv", TrainY_drop_obj, delimiter=",")
np.savetxt("Xtrain_drop_obj.csv", Xtrain, delimiter=",")
np.savetxt("Xtest_drop_obj.csv", Xtest, delimiter=",")

In [ ]:
#func: '6.300000e+001.1' to '6.300000e+001', right-hand string cutting 
def cut_str(mystr):
  for i in range(-1,-15,-1):
    if mystr[i] == 'e':
      return float(mystr)
    if mystr[i] == '.': 
      return float(mystr[0:len(mystr)+i])

In [ ]:
import pandas as pd
import numpy as np 

dfTrainX = pd.read_csv('Xtrain_drop_obj.csv')
dfTrainY = pd.read_csv('Ytrain_drop_obj.csv')
dfTest = pd.read_csv('Xtest_drop_obj.csv') 

firstrowdfTest = []
for el1 in dfTest.columns.to_list():
  firstrowdfTest.append(cut_str(el1))  
firstrowdfTest = np.array(firstrowdfTest)
firstrowdfTrainX = []
for el2 in dfTrainX.columns.to_list():
  firstrowdfTrainX.append(cut_str(el2))
firstrowdfTrainX = np.array(firstrowdfTrainX)
firstrowdfTrainY = dfTrainY.columns.astype(np.float64).to_list()
firstrowdfTrainY = np.array(firstrowdfTrainY)

TrainY = dfTrainY.values
TrainX = dfTrainX.values
TestX = dfTest.values

TrainX = np.insert(TrainX, 0, firstrowdfTrainX, axis=0)
TestX = np.insert(TestX, 0, firstrowdfTest, axis=0)
TrainY = np.insert(TrainY, 0, firstrowdfTrainY, axis=0)
TrainY = TrainY.reshape(24376,)

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor

regr = BaggingRegressor(base_estimator=SVR(),
                        n_estimators=10, random_state=0).fit(TrainX, TrainY)
predicted_data = regr.predict(TestX)